### 第一个Agent

In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain import hub
from langchain.agents import load_tools, create_openai_functions_agent, AgentExecutor

llm=ChatTongyi(model="qwen-plus")

# 定义agent的prompt
# https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")

# 定义工具，加载预制的工具，注意有的工具需要提供llm
tools = load_tools(["llm-math"], llm=llm)

# 创建agent
agent = create_openai_functions_agent(llm,tools,prompt)

# 定义agent的执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

d:\My Documents\Desktop\langchain\.venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [3]:
agent_executor.invoke({"input":"你好"})



> Entering new AgentExecutor chain...
你好！今天过得怎么样？

> Finished chain.


{'input': '你好', 'output': '你好！今天过得怎么样？'}

### Agent
- 中间步骤处理
- 提示词
- 模型配置（停止符必要的话）
- 输出解析器
---

In [15]:
from langchain import hub
from langchain.agents import AgentExecutor, tool
from langchain.agents.output_parsers import XMLAgentOutputParser
from langchain_community.chat_models.tongyi import ChatTongyi

llm=ChatTongyi(model="qwen-plus")

@tool
def search(query:str) ->str:
    """当需要了解最新的天气信息时，使用这个工具"""
    return "晴朗，32摄氏度。无风"

tool_list=[search]

In [16]:
# 提示词模板
prompt = hub.pull("hwchase17/xml-agent-convo")

prompt

d:\My Documents\Desktop\langchain\.venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tools'], input_types={}, partial_variables={'chat_history': ''}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], input_types={}, partial_variables={}, template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\n

In [18]:
# 中间步骤，实现一个log
def convert_intermediate_steps(intermediate_steps):
    log=""
    for action, observation in intermediate_steps:
        log += (
            f"<tool>{action.tool}</tool>"
            f"<tool_input>{action.tool_input}</tool_input>"
            f"<observation>{observation}</observation>"
        )
    return log

In [19]:
# 将工具列表插入到模板中
def convert_tools(tools):
    return "\n".join([f"{tool.name}:{tool.description}" for tool in tools])

In [20]:
# 定义agent
agent=(
    {
        "input":lambda x : x["input"],
        "agent_scratchpad":lambda x : convert_intermediate_steps(x["intermediate_steps"])
    }
    | prompt.partial(tools=convert_tools(tool_list))
    | llm.bind(stop=["</tool_input>","</final_answer>"])
    | XMLAgentOutputParser()
)

In [21]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'chat_history': '', 'tools': 'search:当需要了解最新的天气信息时，使用这个工具'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], input_types={}, partial_variables={}, template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respo

In [23]:
agent_executor=AgentExecutor(
    agent=agent,
    tools=tool_list,
    verbose=True
)

In [25]:
agent_executor.invoke({"input":"深圳今天天气怎么样？"})



> Entering new AgentExecutor chain...
<tool>search</tool><tool_input>深圳 今天 天气晴朗，32摄氏度。无风<final_answer>深圳今天的天气是晴朗，温度为32摄氏度，无风。

> Finished chain.


{'input': '深圳今天天气怎么样？', 'output': '深圳今天的天气是晴朗，温度为32摄氏度，无风。'}

### 一个比较复杂的agent
- 工具
- 检索增强RAG
- 记忆

In [41]:
from langchain.pydantic_v1 import BaseModel,Field
from langchain.tools import BaseTool,StructuredTool,tool
from langchain_community.utilities import SerpAPIWrapper

In [42]:
@tool
def search(query:str) -> str:
    """当需要查找实时信息时会使用这个工具"""
    serp=SerpAPIWrapper()
    return serp.run(query)

print(search.name)
print(search.description)
print(search.args)

search
当需要查找实时信息时会使用这个工具
{'query': {'title': 'Query', 'type': 'string'}}


In [29]:
search("武汉天气怎么样?")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_24400\2017367614.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search("武汉天气怎么样?")


"{'type': 'weather_result', 'temperature': '91', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '59%', 'wind': '2 mph', 'location': '中国湖北省武汉市', 'date': 'Friday 5:00 PM', 'weather': 'Mostly cloudy'}"

In [43]:
# 增强生成

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.embeddings import DashScopeEmbeddings

loader=WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs=loader.load()

print(f"Loaded {len(docs)} documents")

documents=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents,DashScopeEmbeddings())

retriever=vector.as_retriever()



Loaded 1 documents


In [44]:
retriever.get_relevant_documents("how to debug?")[0]

Document(id='7adcb040-c42d-45bd-b5aa-ae8480f402a2', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': '\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'language': 'en'}, page_content='\uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2025 LangChain, Inc.')

In [45]:
# 把检索器加入工具中

from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(
    retriever,
    "langsmith_search",
    """搜索有关LangSmith的信息。关于LangSmith的任何问题，你一定要使用这个工具！"""
)

retriever_tool

Tool(name='langsmith_search', description='搜索有关LangSmith的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000139DFFE5CF0>, retriever=VectorStoreRetriever(tags=['FAISS', 'DashScopeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000139D4194430>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000139DFFE5B40>, retriever=VectorStoreRetriever(tags=['FAISS', 'DashScopeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000139D4194430>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'),

In [46]:
# 可用工具集
tools=[search, retriever_tool]

In [47]:
# 定义模型

llm=ChatTongyi(model="qwen-plus",tempereture=0)

In [48]:
# 从hub中获取模板

from langchain import hub

prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

d:\My Documents\Desktop\langchain\.venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [49]:
# 创建agent

from langchain.agents import create_openai_functions_agent

agent=create_openai_functions_agent(llm,tools,prompt)

In [50]:
# 创建agent执行器

from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [51]:
# 执行
agent_executor.invoke({"input":"langsmith如何帮助做项目测试？"})



> Entering new AgentExecutor chain...


ValueError: request_id: bc5697f6-9e05-9350-8ffb-f46e4a4767e5 
 status_code: 400 
 code: InvalidParameter 
 message: <400> InternalError.Algo.InvalidParameter: Tool names are not allowed to be [search]

In [53]:
# 交互时增加记忆
agent_executor.invoke({
    "input":"hi,my name is Mach.",
    "chat_history":[]
})



> Entering new AgentExecutor chain...


ValueError: request_id: 9726587d-63fe-9b81-bc72-ec30efd5949d 
 status_code: 400 
 code: InvalidParameter 
 message: <400> InternalError.Algo.InvalidParameter: Tool names are not allowed to be [search]